In [14]:
import sys
sys.path.append('../metropolis_within_gibbs/')
from soiaporn_functions import *
from fancy import Data
import pystan

In [19]:
# load data
uhecr_file = 'data/augerUHECR_data2010.dat'
source_file = 'data/agn_catalog.dat'

data = Data()
data.add_source(source_file, 'AGN')
data.add_uhecr(uhecr_file, 'Auger2010')

N_C = len(data.uhecr.energy)
theta = np.deg2rad(data.uhecr.incidence_angle)
d = data.uhecr.unit_vector
varpi = data.source.unit_vector
D = data.source.distance

# hyperparametrs
a = 1 
b = 1
s = 0.01 * 4 * np.pi


F_T = 0.2
f = 0.1

my_table_file = 'data/tableforfig4.data.R'
eps = np.transpose(pystan.read_rdump(my_table_file)['table'])
alpha_T = 20370
M = 7.42
eps = eps * alpha_T / M

In [17]:
kappa = 100 
kappa_c = 1000
a = fik(kappa, kappa_c, d[0], varpi[0], theta[0])
b = np.exp(log_fik(kappa, kappa_c, d[0], varpi[0], theta[0]))
print(a)
print(b)

nan
2.123536356245786e-50


metropolis_within_gibbs/soiaporn_functions.py:49: RuntimeWarning: overflow encountered in sinh
  term1 = kappa * kappa_c / (4 * np.pi * np.sinh(kappa) * np.sinh(kappa_c))
metropolis_within_gibbs/soiaporn_functions.py:51: RuntimeWarning: overflow encountered in sinh
  term2 = np.sinh(inner) / inner
metropolis_within_gibbs/soiaporn_functions.py:52: RuntimeWarning: invalid value encountered in double_scalars
  return A * np.cos(theta_i) * term1 * term2


In [28]:
w = get_weights(D)
lam = []
for i in range(N_C):
    p = get_p_lam(f, eps[2], kappa, kappa_c, d[i], theta[i], varpi, w)
    sample = np.asarray(np.random.multinomial(1, p))
    lam.append(np.where(sample == 1)[0][0])

In [31]:
print(lam.count(0)/ N_C)

0.9565217391304348


In [38]:
c = np.exp(log_p_f(F_T, 0.1, eps[2], lam, w, N_C, a, b))
print(c)

0.0
